# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
file = 'city_data_df.csv'
data = pd.read_csv(file)
data


,City Name,Lat,Lng,Max Temp,Humidity,Clouds,Wind,Country,Date
0,hilo,19.7297,-155.0900,83.93,78,40,8.05,US,1628726909
1,tura,25.5198,90.2201,75.06,98,100,6.82,IN,1628726909
2,mataura,-46.1927,168.8643,57.47,82,100,6.08,NZ,1628726910
3,yellowknife,62.4560,-114.3525,56.80,47,100,11.21,CA,1628726910
4,atuona,-9.8000,-139.0333,77.74,67,28,17.76,PF,1628726911
...,...,...,...,...,...,...,...,...,...
559,yanam,16.7333,82.2167,81.73,70,98,5.26,IN,1628727010
560,choix,26.7092,-108.3219,89.94,56,80,3.56,MX,1628727171
561,olivenza,38.6832,-7.1030,83.21,47,0,4.18,ES,1628727172
562,zhezkazgan,47.8043,67.7144,63.37,55,18,11.61,KZ,1628727134


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)
fig = gmaps.figure()
fig

Figure(layout=FigureLayout(height='420px'))

In [4]:
gmaps.configure(api_key=g_key)
locations = data[['Lat', 'Lng']]
humidity = data['Humidity']
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights = humidity, 
                                 dissipating=False, max_intensity=100,
                                point_radius=3)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
hotel_df = data.loc[(data['Max Temp'] < 80) & (data['Max Temp'] > 70) & (data['Wind'] < 10) & (data['Clouds'] == 0)].copy()

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:

hotel_df['Hotel Name'] = ''

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {
        "radius": 5000,
        "type": "hotel",
        "key": g_key}

for index, row in hotel_df.iterrows():
    
    params['location'] = f"{row['Lat']}, {row['Lng']}"


    response = requests.get(base_url ,params=params)
    results = response.json()
    
    hotel_df.loc[index, 'Hotel Name']= results['results'][0]['name']
hotel_df.rename(columns={'City Name': 'City'}, inplace=True)

hotel_df

,City,Lat,Lng,Max Temp,Humidity,Clouds,Wind,Country,Date,Hotel Name
8,melfi,40.9965,15.6514,73.22,53,0,3.18,IT,1628726912,Melfi
164,cayenne,4.9333,-52.3333,78.10,86,0,0.00,GF,1628726679,Cayenne
275,saint-francois,46.4154,3.9054,70.18,94,0,4.61,FR,1628727027,Molinet
303,hami,42.8000,93.4500,77.88,33,0,6.60,CN,1628727039,Hami
516,bucak,37.4592,30.5950,77.04,54,0,7.29,TR,1628727150,Bucak


In [7]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [8]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations)
fig.add_layer(markers)
# Display figure
fig

Figure(layout=FigureLayout(height='420px'))